# Customer Churn Prediction with Amazon SageMaker Autopilot
_**Using AutoPilot to Predict Mobile Customer Departure**_

---

---

Kernel `Python 3 (Data Science)` works well with this notebook.

## Contents

1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Settingup)
1. [Autopilot Results](#Results)
1. [Host](#Host)
1. [Cleanup](#Cleanup)


---

## Introduction

Amazon SageMaker Autopilot is an automated machine learning (commonly referred to as AutoML) solution for tabular datasets. You can use SageMaker Autopilot in different ways: on autopilot (hence the name) or with human guidance, without code through SageMaker Studio, or using the AWS SDKs. This notebook, as a first glimpse, will use the AWS SDKs to simply create and deploy a machine learning model.

Losing customers is costly for any business.  Identifying unhappy customers early on gives you a chance to offer them incentives to stay.  This notebook describes using machine learning (ML) for the automated identification of unhappy customers, also known as customer churn prediction. ML models rarely give perfect predictions though, so this notebook is also about how to incorporate the relative costs of prediction mistakes when determining the financial outcome of using ML.

We use an example of churn that is familiar to all of us–leaving a mobile phone operator.  Seems like I can always find fault with my provider du jour! And if my provider knows that I’m thinking of leaving, it can offer timely incentives–I can always use a phone upgrade or perhaps have a new feature activated–and I might just stick around. Incentives are often much more cost effective than losing and reacquiring a customer.

---
## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [1]:
import sagemaker
import boto3
from sagemaker import get_execution_role

region = boto3.Session().region_name

session = sagemaker.Session()

# You can modify the following to use a bucket of your choosing
bucket = session.default_bucket()
prefix = 'cchFirstCredit/DEMO-autopilot'

role = get_execution_role()

# This is the client we will use to interact with SageMaker AutoPilot
sm = boto3.Session().client(service_name='sagemaker',region_name=region)

Next, we'll import the Python libraries we'll need for the remainder of the exercise.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import sagemaker
from sagemaker.predictor import csv_serializer

---
## Data

Mobile operators have historical records on which customers ultimately ended up churning and which continued using the service. We can use this historical information to construct an ML model of one mobile operator’s churn using a process called training. After training the model, we can pass the profile information of an arbitrary customer (the same profile information that we used to train the model) to the model, and have the model predict whether this customer is going to churn. Of course, we expect the model to make mistakes–after all, predicting the future is tricky business! But I’ll also show how to deal with prediction errors.

The dataset we will use is synthetically generated, but indictive of the types of features you'd see in this use case.

In [ ]:
#!aws s3 cp s3://sagemaker-sample-files/datasets/tabular/synthetic/churn.txt ./
# only have to run once
#!aws s3 cp s3://XX/datascience/dataset/20200415/directSalecClean/xtest.csv xtest.csv
#!aws s3 cp s3://XX/datascience/dataset/20200415/directSalecClean/xtrain.csv xtrain.csv    

### Upload the dataset to S3

Before you run Autopilot on the dataset, first perform a check of the dataset to make sure that it has no obvious errors. The Autopilot process can take long time, and it's generally a good practice to inspect the dataset before you start a job. This particular dataset is small, so you can inspect it in the notebook instance itself. If you have a larger dataset that will not fit in a notebook instance memory, inspect the dataset offline using a big data analytics tool like Apache Spark. [Deequ](https://github.com/awslabs/deequ) is a library built on top of Apache Spark that can be helpful for performing checks on large datasets. Autopilot is capable of handling datasets up to 5 GB.

Read the data into a Pandas data frame and take a look.

In [5]:
pd.set_option('display.max_columns', 500)
# read in the csv file
directSaleTrain = pd.read_csv('../xtrain.csv')
directSaleTest = pd.read_csv('../xtest.csv')
directSale = pd.concat([directSaleTrain, directSaleTest], axis=0).reset_index()

# print out some data
print('Data shape (rows, cols): ', directSale.shape)
print()
directSale.head()

Data shape (rows, cols):  (17719, 191)



,index,maxmora,cl_unq_act_act_fnacimiento,cl_unq_act_act_finicionegocio,cl_unq_act_act_fechasolicitud,numero_cuotas,cl_unq_act_act_personasdependientes,cl_unq_act_act_plazosolicitado,cl_unq_act_act_diasolicitado,cl_unq_act_act_tiempoentrecompras,cl_unq_act_act_cantidadbanios,cl_unq_act_act_cantidaddormitorios,cl_unq_act_act_cantidadniveles,cl_unq_act_act_totalingresosfam,cl_unq_act_act_negociomontoventasefectivo,cl_unq_act_act_latitud,cl_unq_act_act_promediobuenasventas,cl_unq_act_act_promediomalasventas,cl_unq_act_act_montoenefectivoactual,cl_unq_act_act_estresingresosfamilia,cl_unq_act_act_empresamontodegastostotales,cl_unq_act_act_estresgananciaajustada,cl_unq_act_act_herencia,cl_unq_act_act_ventasbuenlunes,cl_unq_act_act_ventasmallunes,cl_unq_act_act_ventasmalsabado,cl_unq_act_act_negocioingresos,cl_unq_act_act_negociomontocomprascredito,cl_unq_act_act_ventasbuenmiercoles,cl_unq_act_act_totalgastosfam,cl_unq_act_act_negociomontoventascredito,cl_unq_act_act_ventasbuenviernes,cl_unq_act_act_totalbienesfijos,cl_unq_act_act_totalbienes,cl_unq_act_act_ventasmalmartes,cl_unq_act_act_altitud,cl_unq_act_act_estresgrossprofit,cl_unq_act_act_longitud,cl_unq_act_act_estrestotalingresosfamilia,cl_unq_act_act_ventasbuensabado,cl_unq_act_act_negociototalingresos,cl_unq_act_act_montosolicitado,cl_unq_act_act_estresventas,cl_unq_act_act_ventasmalviernes,cl_unq_act_act_payablefee,cl_unq_act_act_ventasmaldomingo,cl_unq_act_act_totalpasivos,cl_unq_act_act_negociomontocomprasefectivo,cl_unq_act_act_accuracy,cl_unq_act_act_ventasmaljueves,cl_unq_act_act_ventasbuenjueves,cl_unq_act_act_estresotrosingresos,cl_unq_act_act_estrescostoventas,cl_unq_act_act_currenttotal,cl_unq_act_act_ventasbuendomingo,cl_unq_act_act_ventasbuenmartes,cl_unq_act_act_totalgastosfamcomida,cl_unq_act_act_ventasmalmiercoles,cl_unq_act_act_estresfamiliacostos,cl_unq_act_act_negocioaproxegresos,cl_unq_act_act_depnacimiento,cl_unq_act_act_genero,cl_unq_act_act_estadocivil,cl_unq_act_act_estadocivilmodificado,cl_unq_act_act_profesion,cl_unq_act_act_nivelacademico,cl_unq_act_act_tipovivienda,cl_unq_act_act_topografia,cl_unq_act_act_tipoaccesovehicular,cl_unq_act_act_tipoaccesopeatonal,cl_unq_act_act_vivtipoconstruccion,cl_unq_act_act_fuenteingresos,cl_unq_act_act_cargo,cl_unq_act_act_depnegocio,cl_unq_act_act_tiponegocio,cl_unq_act_act_negocioactividad,cl_unq_act_act_proveedores,cl_unq_act_act_tiempovivirresidencia,cl_unq_act_act_tipolocalidad,cl_unq_act_act_profesionmodificada,cl_unq_act_act_negociohorasatencion,cl_unq_act_act_detalleventasenero,cl_unq_act_act_detalleventasfebrero,cl_unq_act_act_detalleventasmarzo,cl_unq_act_act_detalleventasabril,cl_unq_act_act_detalleventasmayo,cl_unq_act_act_detalleventasjunio,cl_unq_act_act_detalleventasjulio,cl_unq_act_act_detalleventasagosto,cl_unq_act_act_detalleventasseptiembre,cl_unq_act_act_detalleventasoctubre,cl_unq_act_act_detalleventasnoviembre,cl_unq_act_act_detalleventasdiciembre,cl_unq_act_act_agencia,cl_unq_act_act_division,cl_unq_act_act_region,cl_unq_act_act_campania,cl_unq_act_act_tipoaplicacion,cl_unq_act_act_tipoproducto,cl_unq_act_act_destinofondos,cl_unq_act_act_destinofondosdescripcion,cl_unq_act_act_flagpuedeescribir,cl_unq_act_act_flagpuedeleer,cl_unq_act_act_flagcambiadodireccion,cl_unq_act_act_flaghablaespaniol,cl_unq_act_act_flagpuedefirmar,cl_unq_act_act_flaghablaotroidioma,cl_unq_act_act_flagaccesomensajeros,cl_unq_act_act_flagpidenimpuesto,cl_unq_act_act_flagtienesala,cl_unq_act_act_flagtienecocina,cl_unq_act_act_flagtienecomedor,cl_unq_act_act_flagtienejardin,cl_unq_act_act_flagtienegarage,cl_unq_act_act_flagtieneelectricidad,cl_unq_act_act_flagtieneagua,cl_unq_act_act_flagtienetelfijo,cl_unq_act_act_flagtienecelular,cl_unq_act_act_flagtienetvcable,cl_unq_act_act_flagtienerefrigerador,cl_unq_act_act_flagtienelavadora,cl_unq_act_act_flagtienesecadora,cl_unq_act_act_flagtienehorno,cl_unq_act_act_flagtienemicroondas,cl_unq_act_act_flagtienestereo,cl_unq_act_act_flagvendealcredito,cl_unq_act_act_flagvendeenefec

In [11]:
TARGET = ['maxmora']
date_var = ['cl_unq_act_act_finicionegocio', 'cl_unq_act_act_fechasolicitud', 'index']
FEATURES = [feat for feat in directSaleTrain.columns.to_list() if feat not in date_var]

data = directSale.copy().drop(columns=date_var)

### Reserve some data for calling inference on the model

Divide the data into training and testing splits. The training split is used by SageMaker Autopilot. The testing split is reserved to perform inference using the suggested model.

In [12]:
train_data = data.sample(frac=0.8,random_state=200)

test_data = data.drop(train_data.index)

test_data_no_target = test_data.drop(columns=['maxmora'])

Now we'll upload these files to S3.

In [14]:
train_file = 'train_data.csv';
train_data.to_csv(train_file, index=False, header=True)
train_data_s3_path = session.upload_data(path=train_file, key_prefix=prefix + "/train")
print('Train data uploaded to: ' + train_data_s3_path)

test_file = 'test_data.csv';
test_data_no_target.to_csv(test_file, index=False, header=False)
test_data_s3_path = session.upload_data(path=test_file, key_prefix=prefix + "/test")
print('Test data uploaded to: ' + test_data_s3_path)

Train data uploaded to: s3://sagemaker-us-east-1-475165763880/cchFirstCredit/DEMO-autopilot/train/train_data.csv
Test data uploaded to: s3://sagemaker-us-east-1-475165763880/cchFirstCredit/DEMO-autopilot/test/test_data.csv


---
## Setting up the SageMaker Autopilot Job<a name="Settingup"></a>

After uploading the dataset to Amazon S3, you can invoke Autopilot to find the best ML pipeline to train a model on this dataset. 

The required inputs for invoking a Autopilot job are:
* Amazon S3 location for input dataset and for all output artifacts
* Name of the column of the dataset you want to predict (`maxmora` in this case) 
* An IAM role

Currently Autopilot supports only tabular datasets in CSV format. Either all files should have a header row, or the first file of the dataset, when sorted in alphabetical/lexical order by name, is expected to have a header row.

In [17]:
input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': 's3://{}/{}/train'.format(bucket,prefix)
        }
      },
      'TargetAttributeName': TARGET[0]
    }
  ]

output_data_config = {
    'S3OutputPath': 's3://{}/{}/output'.format(bucket,prefix)
  }

You can also specify the type of problem you want to solve with your dataset (`Regression, MulticlassClassification, BinaryClassification`). In case you are not sure, SageMaker Autopilot will infer the problem type based on statistics of the target column (the column you want to predict). 

Because the target attribute, ```maxmora```, is binary, our model will be performing binary prediction, also known as binary classification. In this example we will let AutoPilot infer the type of problem for us.

You have the option to limit the running time of a SageMaker Autopilot job by providing either the maximum number of pipeline evaluations or candidates (one pipeline evaluation is called a `Candidate` because it generates a candidate model) or providing the total time allocated for the overall Autopilot job. Under default settings, this job takes about four hours to run. This varies between runs because of the nature of the exploratory process Autopilot uses to find optimal training parameters.

### Launching the SageMaker Autopilot Job<a name="Launching"></a>

You can now launch the Autopilot job by calling the `create_auto_ml_job` API. We limit the number of candidates to 20 so that the job finishes in a few minutes.

In [18]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-firstcredit-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      AutoMLJobConfig={'CompletionCriteria':
                                       {'MaxCandidates': 50}
                                      },
                      RoleArn=role)

AutoMLJobName: automl-firstcredit-15-23-09-36


{'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:475165763880:automl-job/automl-firstcredit-15-23-09-36',
 'ResponseMetadata': {'RequestId': '7efddc11-fda7-416b-84ff-b47609536f50',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7efddc11-fda7-416b-84ff-b47609536f50',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '101',
   'date': 'Thu, 15 Apr 2021 23:09:38 GMT'},
  'RetryAttempts': 0}}

In [19]:
#Store the AutoMLJobName name for use in subsequent notebooks 
%store auto_ml_job_name
auto_ml_job_name

Stored 'auto_ml_job_name' (str)


'automl-firstcredit-15-23-09-36'

### Tracking SageMaker Autopilot job progress<a name="Tracking"></a>
SageMaker Autopilot job consists of the following high-level steps : 
* Analyzing Data, where the dataset is analyzed and Autopilot comes up with a list of ML pipelines that should be tried out on the dataset. The dataset is also split into train and validation sets.
* Feature Engineering, where Autopilot performs feature transformation on individual features of the dataset as well as at an aggregate level.
* Model Tuning, where the top performing pipeline is selected along with the optimal hyperparameters for the training algorithm (the last stage of the pipeline). 

In [20]:
print ('JobStatus - Secondary Status')
print('------------------------------')


describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
job_run_status = describe_response['AutoMLJobStatus']
    
while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response['AutoMLJobStatus']
    
    print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
    sleep(30)

JobStatus - Secondary Status
------------------------------
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
I

**AutoPilot automatically generates two executable Jupyter Notebooks:  <br> - SageMakerAutopilotDataExplorationNotebook.ipynb <br> and <br> - SageMakerAutopilotCandidateDefinitionNotebook.ipynb. <br> These notebooks are stored in S3. Let us download them onto our SageMaker Notebook instance so we could explore them later.** 

In [22]:
#print(describe_response)
print(describe_response['AutoMLJobArtifacts']['CandidateDefinitionNotebookLocation'])
print(describe_response['AutoMLJobArtifacts']['DataExplorationNotebookLocation'])

candidate_nbk = describe_response['AutoMLJobArtifacts']['CandidateDefinitionNotebookLocation']
data_explore_nbk = describe_response['AutoMLJobArtifacts']['DataExplorationNotebookLocation']



s3://sagemaker-us-east-1-475165763880/cchFirstCredit/DEMO-autopilot/output/automl-firstcredit-15-23-09-36/sagemaker-automl-candidates/pr-1-bc5970ad572f4d8db2356be60ff5eafe7b0d957035bb4d0f85aae61a2b/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb
s3://sagemaker-us-east-1-475165763880/cchFirstCredit/DEMO-autopilot/output/automl-firstcredit-15-23-09-36/sagemaker-automl-candidates/pr-1-bc5970ad572f4d8db2356be60ff5eafe7b0d957035bb4d0f85aae61a2b/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb


In [23]:
def split_s3_path(s3_path):
    path_parts=s3_path.replace("s3://","").split("/")
    bucket=path_parts.pop(0)
    key="/".join(path_parts)
    return bucket, key

s3_bucket, candidate_nbk_key = split_s3_path(candidate_nbk)
_, data_explore_nbk_key = split_s3_path(data_explore_nbk)

print(s3_bucket, candidate_nbk_key, data_explore_nbk_key)

session.download_data(path='./', bucket=s3_bucket, 
                                 key_prefix = candidate_nbk_key)

session.download_data(path='./', bucket=s3_bucket, 
                                 key_prefix = data_explore_nbk_key)


sagemaker-us-east-1-475165763880 cchFirstCredit/DEMO-autopilot/output/automl-firstcredit-15-23-09-36/sagemaker-automl-candidates/pr-1-bc5970ad572f4d8db2356be60ff5eafe7b0d957035bb4d0f85aae61a2b/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb cchFirstCredit/DEMO-autopilot/output/automl-firstcredit-15-23-09-36/sagemaker-automl-candidates/pr-1-bc5970ad572f4d8db2356be60ff5eafe7b0d957035bb4d0f85aae61a2b/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb


---
## Results

Now use the describe_auto_ml_job API to look up the best candidate selected by the SageMaker Autopilot job. 

In [24]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_name = best_candidate['CandidateName']
print(best_candidate)
print('\n')
print("CandidateName: " + best_candidate_name)
print("FinalAutoMLJobObjectiveMetricName: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("FinalAutoMLJobObjectiveMetricValue: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

{'CandidateName': 'tuning-job-1-fdbc5cd9288447fb8a-038-7673c938', 'FinalAutoMLJobObjectiveMetric': {'MetricName': 'validation:f1', 'Value': 0.6877400279045105}, 'ObjectiveStatus': 'Succeeded', 'CandidateSteps': [{'CandidateStepType': 'AWS::SageMaker::ProcessingJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:475165763880:processing-job/db-1-f81b0ddcddc74b71bfc1bdb21c3660787eadf4bc3e384ae4afe88262d9', 'CandidateStepName': 'db-1-f81b0ddcddc74b71bfc1bdb21c3660787eadf4bc3e384ae4afe88262d9'}, {'CandidateStepType': 'AWS::SageMaker::TrainingJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:475165763880:training-job/automl-fir-dpp0-1-98d26a437fb94be79b448d9413c616fd520a09a7c43f4', 'CandidateStepName': 'automl-fir-dpp0-1-98d26a437fb94be79b448d9413c616fd520a09a7c43f4'}, {'CandidateStepType': 'AWS::SageMaker::TransformJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:475165763880:transform-job/automl-fir-dpp0-csv-1-70326994051b411a91ccb8c6e4342cf2012efcc38', 'CandidateStepName': '

Due to some randomness in the algorithms involved, different runs will provide slightly different results, but accuracy will be around or above $69\%$, which is a good result for a first run.

**If you are curious to explore the performance of other algorithms that AutoPilot explored, they are enumerated for you below via list_candidates_for_auto_ml_job() API call**

In [25]:
#sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
#sm.list_auto_ml_jobs()
sm_dict =sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name)

In [26]:
for item in sm_dict['Candidates']:
    print(item['CandidateName'], item['FinalAutoMLJobObjectiveMetric'])
    print(item['InferenceContainers'][1]['Image'], "\n")

tuning-job-1-fdbc5cd9288447fb8a-049-1a43e637 {'MetricName': 'validation:f1', 'Value': 0.6729300022125244}
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3 

tuning-job-1-fdbc5cd9288447fb8a-050-e1dc2add {'MetricName': 'validation:f1', 'Value': 0.6202899813652039}
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3 

tuning-job-1-fdbc5cd9288447fb8a-048-e315d91d {'MetricName': 'validation:f1', 'Value': 0.6825799942016602}
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3 

tuning-job-1-fdbc5cd9288447fb8a-047-86028c7d {'MetricName': 'validation:f1', 'Value': 0.5917199850082397}
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3 

tuning-job-1-fdbc5cd9288447fb8a-046-04b9bf6f {'MetricName': 'validation:f1', 'Value': 0.4485799968242645}
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3 

tuning-job-1-fdbc5cd9288447fb8a-045-95fa50ab {'MetricName': 'validation:f1'

---
## Host

Now that we've trained the algorithm, let's create a model and deploy it to a hosted endpoint.

In [27]:
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())
model_name = best_candidate_name + timestamp_suffix + "-model"
model_arn = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)

epc_name = best_candidate_name + timestamp_suffix + "-epc"
ep_config = sm.create_endpoint_config(EndpointConfigName = epc_name,
                                      ProductionVariants=[{'InstanceType': 'ml.m5.2xlarge',
                                                           'InitialInstanceCount': 1,
                                                           'ModelName': model_name,
                                                           'VariantName': 'main'}])

ep_name = best_candidate_name + timestamp_suffix + "-ep"
create_endpoint_response = sm.create_endpoint(EndpointName=ep_name,
                                              EndpointConfigName=epc_name)

In [28]:
sm.get_waiter('endpoint_in_service').wait(EndpointName=ep_name)

### Evaluate

Now that we have a hosted endpoint running, we can make real-time predictions from our model very easily, simply by making an http POST request.  But first, we'll need to setup serializers and deserializers for passing our `test_data` NumPy arrays to the model behind the endpoint.

In [52]:
from io import StringIO

if (sagemaker.__version__ < '2'):
    from sagemaker.predictor import RealTimePredictor
    from sagemaker.content_types import CONTENT_TYPE_CSV
    predictor = RealTimePredictor(
        endpoint=ep_name,
        sagemaker_session=session,
        content_type=CONTENT_TYPE_CSV,
        accept=CONTENT_TYPE_CSV)

    # Remove the target column from the test data
    test_data_inference = test_data.drop(TARGET[0], axis=1)

    # Obtain predictions from SageMaker endpoint
    prediction = predictor.predict(test_data_inference.to_csv(sep=',', header=False, index=False)).decode('utf-8')

    # Load prediction in pandas and compare to ground truth
    prediction_df = pd.read_csv(StringIO(prediction), header=None)
    accuracy = (test_data.reset_index()[TARGET[0]] == prediction_df[0]).sum() / len(test_data_inference)
    print('Accuracy: {}'.format(accuracy))

else:
    from sagemaker.predictor import Predictor
    from sagemaker.serializers import CSVSerializer
    from sagemaker.deserializers import CSVDeserializer
    predictor = Predictor(
        endpoint_name=ep_name,
        sagemaker_session=session,
        serializer=CSVSerializer(),
        deserializer=CSVDeserializer())
    
    # Remove the target column from the test data
    test_data_inference = test_data.drop(TARGET[0], axis=1)

    # Obtain predictions from SageMaker endpoint
    prediction = predictor.predict(test_data_inference.to_csv(sep=',', header=False, index=False))
    
    # Load prediction in pandas and compare to ground truth
    prediction_df = pd.DataFrame(prediction)
    accuracy = (test_data.reset_index()[TARGET[0]] == prediction_df[0].astype('int64')).sum() / len(test_data_inference)
    print('Accuracy: {}'.format(accuracy))


Accuracy: 0.7739841986455982


In [64]:
y_test = test_data.reset_index()['maxmora'].astype('int32').values
y_pred = prediction_df[0].astype('int32').values

In [76]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(test_data.reset_index()[TARGET[0]], prediction_df[0].astype('int64'))
precision = precision_score(test_data.reset_index()[TARGET[0]], prediction_df[0].astype('int64'), pos_label=1)
recall = recall_score(test_data.reset_index()[TARGET[0]], prediction_df[0].astype('int64'), pos_label=1, average='binary')
f1 = f1_score(test_data.reset_index()[TARGET[0]], prediction_df[0].astype('int64'), pos_label=1)

print('Accuracy: {}'.format(accuracy))
print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
print('F1: {}'.format(f1))

Accuracy: 0.7739841986455982
Precision: 0.5768645357686454
Recall: 0.42017738359201773
F1: 0.48620910840282233


In [89]:
# code to evaluate the endpoint on test data
# returns a variety of model metrics
def evaluate(test_labels, test_preds, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    tpr = tp / (tp + fn)
    tnr = tn / (tn + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # printing a table of metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print("{:<11} {:.3f}".format('TPR:', tpr))
        print("{:<11} {:.3f}".format('TNR:', tnr))
        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy,
            'TPR': tpr, 'TNR': tnr}


In [90]:
evaluate(y_test, y_pred, verbose=True)


prediction (col)     0    1
actual (row)               
0                 2364  278
1                  523  379

Recall:     0.420
Precision:  0.577
Accuracy:   0.774
TPR:        0.420
TNR:        0.895



{'TP': 379,
 'FP': 278,
 'FN': 523,
 'TN': 2364,
 'Precision': 0.5768645357686454,
 'Recall': 0.42017738359201773,
 'Accuracy': 0.7739841986455982,
 'TPR': 0.42017738359201773,
 'TNR': 0.894776684330053}

---
## Cleanup

The Autopilot job creates many underlying artifacts such as dataset splits, preprocessing scripts, or preprocessed data, etc. This code, when un-commented, deletes them. This operation deletes all the generated models and the auto-generated notebooks as well. 

s3 = boto3.resource('s3')
s3_bucket = s3.Bucket(bucket)

print(s3_bucket)
job_outputs_prefix = '{}/output/{}'.format(prefix, auto_ml_job_name)
print(job_outputs_prefix)
#s3_bucket.objects.filter(Prefix=job_outputs_prefix).delete()

Finally, we delete the endpoint and associated resources.

sm.delete_endpoint(EndpointName=ep_name)
sm.delete_endpoint_config(EndpointConfigName=epc_name)
sm.delete_model(ModelName=model_name)